In [1]:
!python -m spacy download ru_core_news_sm
!pip install razdel
!pip install gensim
!pip install catboost
!pip install pymorphy2
!pip install catboost
# !pip install rank_bm25

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 68.2 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for pymorphy3>=1.0.0 from https://files.pythonhosted.org/packages/d7/f9/ffb9afde503dc6bb2361ea79ceaea18138fbcee32aec4c5d8efa49180753/pymorphy3-1.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for docopt-ng>=0.6 from https://files.pythonhosted.org/packages/6c/4a/c3b77fc1a24510b08918b43a473410c0168f6e657118807015f1f1edceea/docopt_ng-0.9.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 51.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via sp

In [2]:
import numpy as np
import pandas as pd
from scipy import sparse
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from catboost import CatBoostRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

import spacy
import gensim.models
import nltk
from nltk.corpus import stopwords
import razdel
import pymorphy2
from bs4 import BeautifulSoup


import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
class BM25(object):
    def __init__(self, b=0.75, k1=1.6):
        self.vectorizer = TfidfVectorizer(norm=None, smooth_idf=False)
        self.b = b
        self.k1 = k1

    def fit(self, X):
        """ Fit IDF to documents X """
        self.vectorizer.fit(X)
        y = super(TfidfVectorizer, self.vectorizer).transform(X)
        self.avdl = y.sum(1).mean()

    def transform(self, q, X):
        """ Calculate BM25 between query q and documents X """
        b, k1, avdl = self.b, self.k1, self.avdl

        # apply CountVectorizer
        X = super(TfidfVectorizer, self.vectorizer).transform(X)
        len_X = X.sum(1).A1
        q, = super(TfidfVectorizer, self.vectorizer).transform([q])
        assert sparse.isspmatrix_csr(q)

        # convert to csc for better column slicing
        X = X.tocsc()[:, q.indices]
        denom = X + (k1 * (1 - b + b * len_X / avdl))[:, None]
        # idf(t) = log [ n / df(t) ] + 1 in sklearn, so it need to be coneverted
        # to idf(t) = log [ n / df(t) ] with minus 1
        idf = self.vectorizer._tfidf.idf_[None, q.indices] - 1.
        numer = X.multiply(np.broadcast_to(idf, X.shape)) * (k1 + 1)                                                          
        return (numer / denom).sum(1).A1


In [4]:
# функция, возвращает векторизированное предложение
def get_vector(model, sentence, vector_size=100):
    sentence_vector = []

    if len(sentence) == 0:
        # Пустые предложения заполним их одним  словом
        token_vector = np.zeros(vector_size)
        sentence_vector.append(token_vector)
    else:
        for token in sentence:
            try:
                token_vector = model.wv[token]
            except KeyError as e:
                # Случай неизвестного слова
                token_vector = np.zeros(vector_size)
            finally:
                sentence_vector.append(token_vector)
    
    return np.mean(sentence_vector, axis=0)


# функция, возвращает векторизированные выборки
def vectorize_train_test(model, X_train, X_test, vector_size=100):
    X_train_vectorized = np.zeros((X_train.shape[0], vector_size))
    for index, sentence in enumerate(X_train):
        X_train_vectorized[index] = get_vector(model, sentence, vector_size)

    X_test_vectorized = np.zeros((X_test.shape[0], vector_size))
    for index, sentence in enumerate(X_test):
        X_test_vectorized[index] = get_vector(model, sentence,  vector_size)

    return X_train_vectorized, X_test_vectorized


# функция, очистка от html разметки
def clean_html_bs4(text_data):
    soup = BeautifulSoup(text_data, 'lxml')
    return soup.get_text()

# функция, очистка от мусора, нормализация и лемматизация
def tokenize(text, stopwords, need_lemmatize=False):
    result = []
    sentences = [item.text for item in razdel.sentenize(str(text))]

    for sentence in sentences:
        text = sentence.lower()
        text = clean_html_bs4(text)
        text = re.sub(r"\s+", ' ', text)

        tokens = [item.text for item in  razdel.tokenize(text)]
        tokens = [re.sub("[^а-яА-Яa-zA-Z]", ' ', item) for item in tokens]

        if need_lemmatize:
            tokens = [analyzer.parse(token)[0].normal_form for token in tokens if token not in stopwords  and ' ' not in token and len(token) > 2]
            tokens = [token for token in tokens if token not in lemmatized_sw]
        tokens = [re.sub(r"ё", "е", token) for token in tokens]
        result.extend(tokens)
    return result

## Чтение датасета

In [5]:
df = pd.read_csv('/kaggle/input/vacancies/data_vacancies.csv')
df.head(5)

,id,custom_position,schedule,salary_from,salary_to,salary_pay_type,offer_education_id,education_name,education_is_base,education_order_num,city_id,list_regions,work_skills,tags_id
0,48202096,Сварщик-сборщик,полный рабочий день,60000,120000,NaN,0,любое,True,0,2,[4],"['сварочные работы', 'сборка изделий по чертеж...",NaN
1,48202097,Сварщик-монтажник,полный рабочий день,60000,120000,NaN,0,любое,True,0,2,[4],"['монтажные работы', 'строительные работы', 'э...",NaN
2,48202098,Слесарь-сборщик,полный рабочий день,60000,80000,NaN,0,любое,True,0,2,[4],"['работа на фрезерных станках', 'слесарный рем...",NaN
3,48202356,Грузчик-упаковщик,частичная занятость,30000,35000,NaN,0,любое,True,0,1,[3],"['комплектация товара', 'маркировка', 'стрессо...","[6, 9]"
4,48202357,Грузчик-упаковщик,частичная занятость,30000,35000,NaN,0,любое,True,0,57,"[181, 182, 183, 185, 186, 187, 188, 189, 190, ...","['маркировка', 'стрессоустойчивость', 'погрузо...","[6, 9]"


In [6]:
df.shape

(19489, 14)

#### Выделение полей

In [7]:
columns = ['custom_position', 'schedule', 'salary_to', 'city_id', 'work_skills']
df = df[columns]
df.head(5)

,custom_position,schedule,salary_to,city_id,work_skills
0,Сварщик-сборщик,полный рабочий день,120000,2,"['сварочные работы', 'сборка изделий по чертеж..."
1,Сварщик-монтажник,полный рабочий день,120000,2,"['монтажные работы', 'строительные работы', 'э..."
2,Слесарь-сборщик,полный рабочий день,80000,2,"['работа на фрезерных станках', 'слесарный рем..."
3,Грузчик-упаковщик,частичная занятость,35000,1,"['комплектация товара', 'маркировка', 'стрессо..."
4,Грузчик-упаковщик,частичная занятость,35000,57,"['маркировка', 'стрессоустойчивость', 'погрузо..."


#### Возьмем вакансии для топ5 локаций

In [8]:
top5_loc = df['city_id'].value_counts()[:5].index.tolist()
df = df[df['city_id'].isin(top5_loc)]
df.head(5)

,custom_position,schedule,salary_to,city_id,work_skills
0,Сварщик-сборщик,полный рабочий день,120000,2,"['сварочные работы', 'сборка изделий по чертеж..."
1,Сварщик-монтажник,полный рабочий день,120000,2,"['монтажные работы', 'строительные работы', 'э..."
2,Слесарь-сборщик,полный рабочий день,80000,2,"['работа на фрезерных станках', 'слесарный рем..."
3,Грузчик-упаковщик,частичная занятость,35000,1,"['комплектация товара', 'маркировка', 'стрессо..."
4,Грузчик-упаковщик,частичная занятость,35000,57,"['маркировка', 'стрессоустойчивость', 'погрузо..."


#### Проверим пропуски

In [9]:
df.isna().sum()

custom_position    0
schedule           0
salary_to          0
city_id            0
work_skills        0
dtype: int64

In [10]:
df.dropna(inplace=True)

In [11]:
df.shape

(18415, 5)

### Обработка полей

#### OneHotEncoding полей city_id и schedule

In [12]:
city_id_one = pd.get_dummies(df['city_id'], prefix='city_id', dtype=int)
schedule_one = pd.get_dummies(df['schedule'], prefix='schedule', dtype=int)

df = pd.concat([df, city_id_one], axis=1)
df = pd.concat([df, schedule_one], axis=1)

df.drop(columns=['city_id'], inplace=True)
df.drop(columns=['schedule'], inplace=True)
df.head(5)

,custom_position,salary_to,work_skills,city_id_1,city_id_2,city_id_57,city_id_102,city_id_174,schedule_вахта,schedule_полный рабочий день,schedule_свободный график,schedule_сменный график,schedule_удаленная работа,schedule_частичная занятость
0,Сварщик-сборщик,120000,"['сварочные работы', 'сборка изделий по чертеж...",0,1,0,0,0,0,1,0,0,0,0
1,Сварщик-монтажник,120000,"['монтажные работы', 'строительные работы', 'э...",0,1,0,0,0,0,1,0,0,0,0
2,Слесарь-сборщик,80000,"['работа на фрезерных станках', 'слесарный рем...",0,1,0,0,0,0,1,0,0,0,0
3,Грузчик-упаковщик,35000,"['комплектация товара', 'маркировка', 'стрессо...",1,0,0,0,0,0,0,0,0,0,1
4,Грузчик-упаковщик,35000,"['маркировка', 'стрессоустойчивость', 'погрузо...",0,0,1,0,0,0,0,0,0,0,1


#### Нормализация и токенизации поля custom_position

In [13]:
nlp = spacy.load('ru_core_news_sm')

df['custom_position_clear'] = df['custom_position'].apply(
    lambda x: ' '.join([str(token).lower() for token in nlp(x) if
      not token.is_stop
      and not token.is_punct
      and not token.is_digit
      and not token.like_email
      and not token.like_num
      and not token.is_space
    ])
  )

df.drop(columns=['custom_position'], inplace=True)
df.head(5)

,salary_to,work_skills,city_id_1,city_id_2,city_id_57,city_id_102,city_id_174,schedule_вахта,schedule_полный рабочий день,schedule_свободный график,schedule_сменный график,schedule_удаленная работа,schedule_частичная занятость,custom_position_clear
0,120000,"['сварочные работы', 'сборка изделий по чертеж...",0,1,0,0,0,0,1,0,0,0,0,сварщик сборщик
1,120000,"['монтажные работы', 'строительные работы', 'э...",0,1,0,0,0,0,1,0,0,0,0,сварщик монтажник
2,80000,"['работа на фрезерных станках', 'слесарный рем...",0,1,0,0,0,0,1,0,0,0,0,слесарь сборщик
3,35000,"['комплектация товара', 'маркировка', 'стрессо...",1,0,0,0,0,0,0,0,0,0,1,грузчик упаковщик
4,35000,"['маркировка', 'стрессоустойчивость', 'погрузо...",0,0,1,0,0,0,0,0,0,0,1,грузчик упаковщик


#### Обработка work_skills

In [14]:
for index, row in df.iterrows():
    df['work_skills'][index] = ', '.join([word.replace('\'', '').lower() 
                                         for word in row['work_skills'].split('[')[1].split(']')[0].split(', ')])
df.head(5)

,salary_to,work_skills,city_id_1,city_id_2,city_id_57,city_id_102,city_id_174,schedule_вахта,schedule_полный рабочий день,schedule_свободный график,schedule_сменный график,schedule_удаленная работа,schedule_частичная занятость,custom_position_clear
0,120000,"сварочные работы, сборка изделий по чертежам, ...",0,1,0,0,0,0,1,0,0,0,0,сварщик сборщик
1,120000,"монтажные работы, строительные работы, электро...",0,1,0,0,0,0,1,0,0,0,0,сварщик монтажник
2,80000,"работа на фрезерных станках, слесарный ремонт,...",0,1,0,0,0,0,1,0,0,0,0,слесарь сборщик
3,35000,"комплектация товара, маркировка, стрессоустойч...",1,0,0,0,0,0,0,0,0,0,1,грузчик упаковщик
4,35000,"маркировка, стрессоустойчивость, погрузочно-ра...",0,0,1,0,0,0,0,0,0,0,1,грузчик упаковщик


#### Дедубликация

In [15]:
columns = list(df.columns)
columns.remove('salary_to')
columns

['work_skills',
 'city_id_1',
 'city_id_2',
 'city_id_57',
 'city_id_102',
 'city_id_174',
 'schedule_вахта',
 'schedule_полный рабочий день',
 'schedule_свободный график',
 'schedule_сменный график',
 'schedule_удаленная работа',
 'schedule_частичная занятость',
 'custom_position_clear']

In [16]:
df_duplicated = df[df[columns].duplicated(keep=False)]
df_duplicated.shape

(2087, 14)

In [17]:
df = df.drop(df[df[columns].duplicated(keep=False)].index)
df.head(5)

,salary_to,work_skills,city_id_1,city_id_2,city_id_57,city_id_102,city_id_174,schedule_вахта,schedule_полный рабочий день,schedule_свободный график,schedule_сменный график,schedule_удаленная работа,schedule_частичная занятость,custom_position_clear
0,120000,"сварочные работы, сборка изделий по чертежам, ...",0,1,0,0,0,0,1,0,0,0,0,сварщик сборщик
1,120000,"монтажные работы, строительные работы, электро...",0,1,0,0,0,0,1,0,0,0,0,сварщик монтажник
2,80000,"работа на фрезерных станках, слесарный ремонт,...",0,1,0,0,0,0,1,0,0,0,0,слесарь сборщик
3,35000,"комплектация товара, маркировка, стрессоустойч...",1,0,0,0,0,0,0,0,0,0,1,грузчик упаковщик
4,35000,"маркировка, стрессоустойчивость, погрузочно-ра...",0,0,1,0,0,0,0,0,0,0,1,грузчик упаковщик


In [18]:
df_deduplicated = pd.DataFrame(df_duplicated.groupby(columns)['salary_to'].median()).reset_index()
df_deduplicated.shape

(673, 14)

In [19]:
df = pd.concat([df.drop(['salary_to'], axis=1), df['salary_to']], axis=1)
df = pd.concat([df, df_deduplicated], axis=0).reset_index(drop=True)

In [20]:
df

,work_skills,city_id_1,city_id_2,city_id_57,city_id_102,city_id_174,schedule_вахта,schedule_полный рабочий день,schedule_свободный график,schedule_сменный график,schedule_удаленная работа,schedule_частичная занятость,custom_position_clear,salary_to
0,"сварочные работы, сборка изделий по чертежам, ...",0,1,0,0,0,0,1,0,0,0,0,сварщик сборщик,120000.0
1,"монтажные работы, строительные работы, электро...",0,1,0,0,0,0,1,0,0,0,0,сварщик монтажник,120000.0
2,"работа на фрезерных станках, слесарный ремонт,...",0,1,0,0,0,0,1,0,0,0,0,слесарь сборщик,80000.0
3,"комплектация товара, маркировка, стрессоустойч...",1,0,0,0,0,0,0,0,0,0,1,грузчик упаковщик,35000.0
4,"маркировка, стрессоустойчивость, погрузочно-ра...",0,0,1,0,0,0,0,0,0,0,1,грузчик упаковщик,35000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16996,электромонтажные работы,0,0,1,0,0,0,1,0,0,0,0,электромонтажник,100000.0
16997,электромонтажные работы,0,1,0,0,0,0,1,0,0,0,0,электромонтажник,100000.0
16998,электромонтажные работы,1,0,0,0,0,0,1,0,0,0,0,макетчик монтажник,200000.0
16999,электромонтажные работы,1,0,0,0,0,0,1,0,0,0,0,электромонтажник,100000.0


#### Векторизация custom_position

Для обработки остальных полей необходимо предварительно разделить на тренировочную и тестовую выборки

In [21]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'salary_to'], df['salary_to'], random_state=2023)

In [22]:
fasttext = gensim.models.FastText(vector_size=100, min_count=1)
fasttext.build_vocab(corpus_iterable=X_train['custom_position_clear'])

fasttext.train(
    corpus_iterable=X_train['custom_position_clear'],
    total_examples=len(X_train['custom_position_clear']),
    epochs=20
    )

(1229324, 6056040)

In [23]:
cuspos_vec_train, cuspos_vec_test = vectorize_train_test(fasttext, X_train['custom_position_clear'], X_test['custom_position_clear'])
cuspos_vec_train = pd.DataFrame(cuspos_vec_train).add_prefix('custom_position_')
cuspos_vec_test = pd.DataFrame(cuspos_vec_test).add_prefix('custom_position_')

In [24]:
X_train = pd.concat([X_train.reset_index(drop=True), cuspos_vec_train], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), cuspos_vec_test], axis=1)

X_train.drop(columns=['custom_position_clear'], inplace=True)
X_test.drop(columns=['custom_position_clear'], inplace=True)

In [25]:
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [26]:
X_train.head(5)

,work_skills,city_id_1,city_id_2,city_id_57,city_id_102,city_id_174,schedule_вахта,schedule_полный рабочий день,schedule_свободный график,schedule_сменный график,...,custom_position_90,custom_position_91,custom_position_92,custom_position_93,custom_position_94,custom_position_95,custom_position_96,custom_position_97,custom_position_98,custom_position_99
0,розничные продажи,0,0,1,0,0,0,0,0,1,...,0.071752,0.094296,-0.279740,-0.024182,-0.124967,-0.019596,0.059302,0.196966,-0.149955,0.021296
1,"детская психология, грамотная речь, работа с д...",1,0,0,0,0,0,1,0,0,...,0.039951,0.008213,-0.351772,-0.218028,-0.120750,0.290850,0.013165,0.143993,-0.241703,0.082178
2,"экспедирование, доставка товаров, перевозка гр...",0,0,0,0,1,0,0,0,1,...,-0.055178,-0.023030,-0.315926,-0.047202,-0.089029,0.111394,0.024267,0.176660,-0.008882,-0.024820
3,"ремонт иномарок, слесарный ремонт, проведение ...",1,0,0,0,0,0,1,0,0,...,-0.107321,-0.040236,-0.292178,-0.010886,-0.090411,0.145146,-0.007261,0.121350,-0.053818,-0.060946
4,"поиск и привлечение клиентов, активные продажи...",1,0,0,0,0,0,1,0,0,...,0.162681,0.003802,-0.286334,-0.089143,0.034394,-0.422194,-0.148483,0.187861,-0.005299,0.205022


#### Векторизация work_skills

In [27]:
sw = stopwords.words('russian')
additional_sw =  'мои оно мной мною мог могут мор мое мочь оба нам нами ними однако нему никуда наш нею неё наша наше наши очень отсюда вон вами ваш ваша ваше ваши весь всем всеми вся ими ею будем будете будешь буду будь будут кому кого которой которого которая которые который которых кем каждое каждая каждые каждый кажется та те тому собой тобой собою тобою тою хотеть хочешь свое свои твой своей своего своих твоя твоё сама сами теми само самом самому самой самого самим самими самих саму чему тебе такое такие также такая сих тех ту эта это этому туда этим этими этих абы аж ан благо буде вроде дабы едва ежели затем зато ибо итак кабы коли коль либо лишь нежели пока покамест покуда поскольку притом причем пускай пусть ровно сиречь словно также точно хотя чисто якобы '
pronouns = 'я мы ты вы он она оно они себя мой твой ваш наш свой его ее их то это тот этот такой таков столько весь всякий сам самый каждый любой иной другой кто что какой каков чей сколько никто ничто некого нечего никакой ничей нисколько кто-то кое-кто кто-нибудь кто-либо что-то кое-что что-нибудь что-либо какой-то какой-либо какой-нибудь некто нечто некоторый некий'
conjunctions = 'что чтобы как когда ибо пока будто словно если потому что оттого что так как так что лишь только как будто с тех пор как в связи с тем что для того чтобы кто как когда который какой где куда откуда'
digits = 'ноль один два три четыре пять шесть семь восемь девять десять одиннадцать двенадцать тринадцать четырнадцать пятнадцать шестнадцать семнадцать восемнадцать девятнадцать двадцать тридцать сорок пятьдесят шестьдесят семьдесят восемьдесят девяносто сто'
modal_words = 'вероятно возможно видимо по-видимому кажется наверное безусловно верно  действительно конечно несомненно разумеется'
particles = 'да так точно ну да не ни неужели ли разве а что ли что за то-то как ну и ведь даже еще ведь уже все все-таки просто прямо вон это вот как словно будто точно как будто вроде как бы именно как раз подлинно ровно лишь только хоть всего исключительно вряд ли едва ли'
prepositions = 'близ  вблизи  вдоль  вокруг  впереди  внутрь  внутри  возле  около  поверх  сверху  сверх  позади  сзади  сквозь  среди  прежде  мимо  вслед  согласно  подобно  навстречу  против  напротив  вопреки  после  кроме  вместе  вдали  наряду  совместно  согласно  нежели вроде от бишь до без аж тех раньше совсем только итак например из прямо ли следствие а поскольку благо пускай благодаря случае затем притом также связи время при чтоб просто того невзирая даром вместо точно покуда тогда зато ради ан буде прежде насчет раз причине тому так даже исходя коль кабы более ровно либо помимо как-то будто если словно лишь бы и не будь пор тоже разве чуть как хотя наряду потому пусть в равно между сверх ибо на судя то чтобы относительно или счет за но сравнению причем оттого есть когда уж ввиду тем для дабы чем хоть с вплоть скоро едва после той да вопреки ежели кроме сиречь же коли под абы несмотря все пока покамест паче прямо-таки перед что по вдруг якобы подобно'
evaluative = 'наиболее наименее лучший больший высший низший худший более менее'

sw.extend(additional_sw.split())
sw.extend(pronouns.split())
sw.extend(conjunctions.split())
sw.extend(digits.split())
sw.extend(modal_words.split())
sw.extend(particles.split())
sw.extend(prepositions.split())
sw.extend(evaluative.split())
sw = list(set(sw))

In [28]:
need_preprocess = True

if need_preprocess:
    analyzer = pymorphy2.MorphAnalyzer()
    lemmatized_sw = [analyzer.parse(word)[0].normal_form for word in sw]
    
    for index, row in X_train.iterrows():
        X_train['work_skills'][index] = tokenize(row['work_skills'], stopwords=sw, need_lemmatize=True)
    for index, row in X_test.iterrows():
        X_test['work_skills'][index] = tokenize(row['work_skills'], stopwords=sw, need_lemmatize=True)

In [29]:
for index, row in X_train.iterrows():
    X_train['work_skills'][index] = " ".join(row['work_skills'])
    
for index, row in X_test.iterrows():
    X_test['work_skills'][index] = " ".join(row['work_skills'])

In [30]:
vectorizer = CountVectorizer(min_df=2, max_features=2000) # max_df фильтрует corpus-specific stop words , 
vectorizer.fit(X_train['work_skills'])
dictionary = vectorizer.get_feature_names_out()

BM25 implementation

In [31]:
bm25 = BM25()
bm25.fit(X_train['work_skills'])

In [32]:
train_embeddings = np.zeros((len(X_train['work_skills']), len(dictionary)))
test_embeddings = np.zeros((len(X_test['work_skills']), len(dictionary)))

for i in range(len(dictionary)):
    doc_scores = bm25.transform(dictionary[i], X_train['work_skills'])
    train_embeddings[:, i] = np.array(doc_scores)

for i in range(len(dictionary)):
    doc_scores = bm25.transform(dictionary[i], X_test['work_skills'])
    test_embeddings[:, i] = np.array(doc_scores)

In [33]:
df_train_embeddings = pd.DataFrame(train_embeddings, columns=dictionary).add_prefix('skill_')
df_test_embeddings = pd.DataFrame(test_embeddings, columns=dictionary).add_prefix('skill_')

In [34]:
X_train_final = pd.concat([X_train, df_train_embeddings], axis=1).drop(['work_skills'], axis=1)
X_test_final = pd.concat([X_test, df_test_embeddings], axis=1).drop(['work_skills'], axis=1)

#### Нормализация данных

In [35]:
scaler = StandardScaler()

X_train_final = scaler.fit_transform(X_train_final)
X_test_final = scaler.transform(X_test_final)

#### Обучение модели

In [36]:
parameters = {'iterations': [100, 1000, 2000], 
              'depth': [2, 4, 6], 
              'learning_rate': [1],
              'task_type': ['GPU'], 
              'devices': ['0:1']}


model = CatBoostRegressor()
cv = GridSearchCV(model, parameters)

In [37]:
cv.fit(X_train_final, y_train)

0:	learn: 55210.0626036	total: 13.5ms	remaining: 1.34s
1:	learn: 53221.8298284	total: 18.7ms	remaining: 917ms
2:	learn: 52728.9038439	total: 24.1ms	remaining: 779ms
3:	learn: 51227.0946975	total: 29.9ms	remaining: 716ms
4:	learn: 50597.7025135	total: 35.4ms	remaining: 672ms
5:	learn: 49887.7801945	total: 40.6ms	remaining: 635ms
6:	learn: 49633.3139888	total: 45.9ms	remaining: 609ms
7:	learn: 49355.4461453	total: 51.2ms	remaining: 589ms
8:	learn: 49176.7780683	total: 56.8ms	remaining: 575ms
9:	learn: 48814.1023445	total: 62.7ms	remaining: 564ms
10:	learn: 48599.8857744	total: 68.2ms	remaining: 552ms
11:	learn: 48494.2807269	total: 73.4ms	remaining: 538ms
12:	learn: 48107.6823196	total: 78.5ms	remaining: 525ms
13:	learn: 47719.0462971	total: 83.9ms	remaining: 515ms
14:	learn: 47386.5769287	total: 89.5ms	remaining: 507ms
15:	learn: 47156.4017475	total: 94.8ms	remaining: 498ms
16:	learn: 46698.7985544	total: 100ms	remaining: 490ms
17:	learn: 46415.5471365	total: 105ms	remaining: 477ms
18:	

GridSearchCV(estimator=<catboost.core.CatBoostRegressor object at 0x7ed8f4feafe0>,
             param_grid={'depth': [2, 4, 6], 'devices': ['0:1'],
                         'iterations': [100, 1000, 2000], 'learning_rate': [1],
                         'task_type': ['GPU']})

In [38]:
cv.best_params_

{'depth': 6,
 'devices': '0:1',
 'iterations': 100,
 'learning_rate': 1,
 'task_type': 'GPU'}

In [39]:
best_model = cv.best_estimator_

In [40]:
y_pred = best_model.predict(X_test_final)
mape = mean_absolute_percentage_error(y_test, y_pred)

print("CatBoostRegressor")
print('mape:', mape)

CatBoostRegressor
mape: 0.3059406537071192


Полный перерасчет tf-idf и bm25 при новом предложении